## Maximum Subarray

* https://leetcode.com/problems/maximum-subarray/
***
* Time Complexity: O(n)
    - must traverse through the entire array to figure out the max
* Space Complexity: O(1)
    - only 2 variables are created and used. no recursion used either
***
* basically you keep a running max and another max that is the highest one you've seen
    - you update the running max as you move along with the current index in mind and then you see if that running max is the biggest you've seen so far and update accordingly

In [1]:
/**
 * @param {number[]} nums
 * @return {number}
 */
var maxSubArray = function(nums) {
    if (nums.length === 1) return nums[0];
    
    let max = nums[0];
    let runningMax = nums[0];
    
    for (let i = 1; i < nums.length; i++) {
        runningMax = Math.max(nums[i] + runningMax, nums[i]);
        max = Math.max(max, runningMax);
    }
    
    return max;
};

## Jump Game

* https://leetcode.com/problems/jump-game/
***
* Time Complexity:
    - naive: O(n$^{n}$)
        * the height of the tree is around n.
            - it will return immediately once it reaches an index past that
        * the number of nodes created from one node is wholly dependent on what the value at i is
            - this is b/c we iterate from [i + 1...i + nums[i]]
            - and nums[i] ~= $10^{5}$ which is an order of magnitude less than the nums length of $10^{4}$ in the constraints section
    - topdown memo: O(n$^{2}$)
        * we remove any overlapping subproblems and only check from [0...n - 1]
    - bottom-up: O(n$^{2}$)
        * similar range for top-down but we also break out of the loop early when we have dp[i] = true
    - greedy: O(n)
        * just traverses through the array once
* Space Complexity:
    - naive: O(n)
        * uses recursion so requires space for the functions in the stack
        * the height of the tree is around n, since it will return from any index greater than that
    - topdown memo:
        * uses recursion so requires space for the function = O(n)
        * needs space for dp table = O(n)
    - bottom-up: 
        * needs space for dp table = O(n)
    - greedy: O(1)
        * only keeps a couple of variables
***
* naive:
    - for each reachable index, we check its range from [i + 1...i + nums[i]] to see if those later indices can reach the last index or past it
    - if index = 0, we return false since we cannot move any further
    - if index >= nums.length - 1, we've reached the last index from the current index
* topdown memo:
    - we use a dp table to keep track of the indices we've seen and know that can reach the last index
* bottom up:
    - we know that the reaching the last index from the last index is trivial and this is our starting point for bottom-up dp
    - as we traverse backwards from the last index, we still look at indices from [i + 1...i + nums[i]] and check 2 things:
        1. can we reach further than the last index?
            * if yes, then dp[i] = true
        2. can we reach an index that can reach the last index?
            * essentially, if we can't reach the last index or further, can we at least reach another index that can?
            * if so, then dp[i] = true;
* greedy:
    - looking at the bottom-up solution, we realize that as long as we can keeping moving forward, we will eventually reach an index that CAN reach the last index
    - we are still implicitly checking the range of [i + 1...i + nums[i]]
        * the loop will carry us forward and check the max index that is reachable at any time
* my greedy:
     - similar structure to Kadane's algorithm
    - my thought process:
        * as long as you can continue traversing through the array, you can move to the last index
        * so as long as your max steps > 0, then you can continue walking
        * so through each iteration
            - we subtract the max to simulate walking to the next index
            - we compare the current max with the number of steps at this index and we take whichever one is larger since it'll give us the best chance to continue walking towards the end
            - if our max steps ever reaches 0, then we've exhausted all of our steps and cannot reach the last index

In [2]:
/**
 * @param {number[]} nums
 * @return {boolean}
 */
var canJump = function(nums) {
  // since you start at the first index,
  // if steps = 0, you might not reach your goal
  // unless you have something like nums = [0]
  if (nums[0] === 0) return nums.length === 1;

  // initialized with first item in array
  let max = nums[0];
  for (let i = 1; i < nums.length - 1; i++) {
    // this is where we make our step
    max--;

    max = Math.max(max, nums[i]);

    // as long as we can keep moving forward, we can reach
    // the last index, so as long as max = 1
    if (max <= 0) return false;

  }

  return true;
};

// naive 
var canJump = function(nums) {
    if (nums[0] === 0) return nums.length === 1;

    const canReach = (index) => {
      
        // can reach last index
        if (index >= nums.length - 1) return true;

        // reached a sinkhole that cannot move further
        if (nums[index] === 0) return false;

        let canReachLast = false;
        for (let i = index + 1; i <= (index + nums[index]); i++) {
            canReachLast ||= canReach(i);
        }

        return canReachLast;
    }

    return canReach(0);
}

// topdown memo
var canJump = function(nums) {
    if (nums[0] === 0) return nums.length === 1;
    const dp = [];

    const canReach = (index) => {
        if (index >= nums.length - 1) return true;
        if (nums[index] === 0) return false;

        if (dp[index] !== undefined) {
          return dp[index];
        }

        let canReachLast = false;
        for (let i = index + 1; i <= (index + nums[index]); i++) {
            canReachLast ||= canReach(i);
        }

        dp[index] = canReachLast;
        return dp[index];
    }

    return canReach(0);
}

// bottom-up dp
var canJump = function(nums) {
  const dp = [];
  
  // the last index can be reached trivially if you're at the last index
  dp[nums.length - 1] = true;

  for (let i = nums.length - 2; i >= 0; i--) {
    for (let j = i + 1; j <= (i + nums[i]); j++) {
        dp[i] ||= (j >= nums.length - 1) || dp[j];

        if (dp[i]) break;
    } 
  }

  return dp[0] !== undefined;
}

// greedy
var canJump = function(nums) {
  let reachableIdx = 0;

  // essentially what we're doing at every index in the bottom-up
  // is seeing how far we can actually reach from that current position
  
  // the inner loop always checks from i + 1 ... (i + nums[i])
  // to see if any of those indices can reach the last index
  
  // as long as our current index can reach any of those later indices
  // then we can reach the last

  // therefore, the reachableIdx acts as a limiter
  // as long as we can keep moving forward
  // we can eventually reach the end
  for (let i = 0; i < nums.length; i++) {
    if (reachableIdx < i) return false;

    reachableIdx = Math.max(reachableIdx, i + nums[i]);
  }

  return true;
}

## Jump Game II

* https://leetcode.com/problems/jump-game-ii/description/
***
* Time Complexity:
    - naive: O(n!)
        * your jumps are limited to the length of the array
        * so if you are at index i, then your max jump is from i + 1...n - 1 or basically n - i
            - therefore, for every node, you have n - i nodes coming out of it
            - thus, this is n * (n - 1) * (n - 2) * (n - 3) ... (n - n) = n!
    - topdown memo: O(n$^{2}$)
        * by memoizing solved subproblems, we reduce the complexity down to O(n$^{2}$)
    - bottom-up: O(n$^{2}$)
        * our outer loop goes from i = n - 2...0 and our inner loop goes from j = i + 1...(i + nums[i])
        * so we are looking at O(n$^{2}$) subproblems basically
    - greedy: O(n)
        * just loops through the entire array once
* Space Complexity:
    - naive: O(n)
        * uses recursion so requires space for the functions in the stack
        * there should be at most O(n) functions in the stack b/c it is bounded by the length of the array, n
        * it will always return when the index reaches n
    - topdown memo: O(n)
        * uses recursion so requires space for the functions in the stack
        * also requres O(n) space for dp table
    - bottom-up: O(n)
        * requires O(n) space for dp table
    - greedy: O(1)
        * only needs space for a couple of variables
***
* naive
    - basically at any index, you can make a jump from [i + 1...i + nums[i]] as long as that range is within the length of nums
    - so at each index, you make those jumps and those jumps can make their own jumps until you reach the end
    - so you're essentially trying out all combinations of jumps and taking their minimum
* topdown memo:
    - the naive algorithm has a lot of overlapping subproblems, especially if the nums array starts off with high values and ends on lower values
        * e.g. [3, 2, 1, 1, 4]
        * at index 0, your range is from [1...3]
        * at index 1, your range is from [2...4]
        * you are repeating the calculations for indices [2,3]
    - therefore, you can use a dp table to return immediately when it's already seen an index
* bottom-up:
    - we know that we can reach the last index from the last index trivially
        * it takes 0 jumps to reach the last index from the last index
    - going off of that case, we want to work our way backwards from that and check the same range from [i + 1...i + nums[i]]
        * so our 2 loops would be i: n - 2...0 and j: i + 1...Math.min(i + nums[i], n - 1)
    - our __recurrence relation__ is heavily based on what we did recursively in our naive/topdown memo solutions:
        * __topdown: minJumps = Math.min(minJumps, 1 + traverse(i))__
        * __bottom-up: dp[i] = Math.min(dp[i], 1 + dp[j]), where j = Math.min(i + nums[i], n - 1) (basically our reach)__
* greedy:
    - our greedy algorithm is basically bfs
    - starting off with the first index, we see how far we can reach from that index
        * this represents exploring the root node and adding the children to the queue
        * all edges from root to child represent 1 jump
    - we then explore the rest of the nodes of that level and update the max reachable index from any of those indices
    - once we've explored all nodes of that level, we increment # of jumps

In [1]:
/**
 * @param {number[]} nums
 * @return {number}
 */

 // naive
var jump = function(nums) {
    const n = nums.length;
    const traverse = (index) => {
        // base case
        // it only takes 0 jumps to reach the last index if you're already
        // at the last index
        if (index === n - 1) return 0;

        // starting at i, you can jump from [i + 1...i + nums[i]]
        // as long as that jump is within the bounds of the array
        const reach = Math.min(n - 1, index + nums[index]);

        let minJumps = Number.POSITIVE_INFINITY;
        for (let i = index + 1; i <= reach; i++) {
            // it's 1 + whatever # of jumps is left to reach the last index
            minJumps = Math.min(minJumps, 1 + traverse(i));
        }

        return minJumps;
    }

    return traverse(0);
};

// topdown memo
var jump = function(nums) {
    const n = nums.length;

    // we're doing a lot of redundant work in our recursion
    // since we're iterating from [i + 1...i + nums[i]]
    // we have lots of opportunities to traverse over the same index again and again
    const dp = [];

    const traverse = (index) => {
        if (index === n - 1) return 0;
        if (dp[index] !== undefined) {
            return dp[index];
        }

        const reach = Math.min(n - 1, index + nums[index]);

        let minJumps = Number.POSITIVE_INFINITY;
        for (let i = index + 1; i <= reach; i++) {
            // it's 1 + whatever # of jumps is left to reach the last index
            minJumps = Math.min(minJumps, 1 + traverse(i));
        }

        dp[index] = minJumps;
        return dp[index];
    }

    return traverse(0);
};

// bottom-up
var jump = function(nums) {
    const n = nums.length;
    const dp = Array.from({ length: n }, () => Number.POSITIVE_INFINITY);

    // base case
    dp[n - 1] = 0;

    // since our base case is at n - 1
    // we want to loop backwards to take advantage of that
    for (let i = nums.length - 2; i >= 0; i--) {

        // pretty much the same as the inside of the topdown memo
        const reach = Math.min(n - 1, i + nums[i]);
        for (let j = i + 1; j <= reach; j++) {

            // since dp[i] = minJumps in topdown
            // our recurrence relation will look like this

            // 1 + dp[j] is 1 + traverse(i)
            // in the topdown memo, i is basically the loop from [i + 1...i + nums[i]]
            // in this loop, j is the same thing
            // they're just called different variables
            dp[i] = Math.min(dp[i], 1 + dp[j]);
        }
    }

    // console.log({dp})
    return dp[0];
}

// greedy
var jump = function(nums) {
    const n = nums.length;

    // think of this like bfs
    // e.g. nums = [2,3,1,1,4]
    // the first level is the first index: 2
    // the second level represents the nodes from i = [i + 1...i + nums[i]] = [1...2] = 3, and 1
    // as you work through the second level, you know that the MAX reachable nodes is at index 1, nums[1] = 3
    // so you're able to reach indices [2...4]
    //          2
    //         / \
    //        3   1
    //      / | \
    //     1  1  4
    // in essence, each level is the MAX reachable nodes from any node in the level before it
    let jumps = 0;
    let reachable = 0;
    let lastJumped = 0;

    for (let i = 0; i < n - 1; i++) {
        // this represents the nodes reachable from the current index
        reachable = Math.max(reachable, i + nums[i]);

        // you can think of each jump as a level in bfs
        // every time you look at all nodes in the current level
        // you update the queue so that it only focuses on the nodes of the next level
        // so you increment jumps
        if (i === lastJumped) {
            lastJumped = reachable;
            jumps++;
        }
    }

    return jumps;
}

## Gas Station

* https://leetcode.com/problems/gas-station/
***
* Time Complexity:
    - naive: O(n$^{2}$)
        * if a gas station's currCost > 0, meaning gas[i] - cost[i] > 0, then we recurse on it
        * our recursion will attempt to go from n...n in a circuit
        * therefore, if every gas station was viable, at most we will see n$^{2}$ subproblems
    - greedy: O(n)
        * we just loop from 1...n which is just O(n)
* Space Complexity:
    - naive: O(n)
        * uses recursion so requires space for functions in the stack
        * the number of functions is bounded by the length of the array since the recursion will stop when there's no more gas or when it reaches the same index it started the circuit on
    - greedy: O(1)
        * only requires space for a couple of variables
***
* naive:
    - manually start the circuit from every index if gas[i] > cost[i] and continue until it reaches back to i
    - for every recursive call, we check if there's any gas left and if the current gas station can supply us the necessary amount to get us to the next station
        * if it doesn't then we move onto the next gas station as the new circuit start
* greedy:
    - algorithm is similar to Maximum Subarray/Kadane's algorithm
    - our first assumption is this:
        * if sum(gas) < sum(cost), then return -1
        * if we don't even have enough gas in total to get through the entire circuit, then there's no point in iterating over the entire thing
    - for the algorithm, we keep track of the totalGas and the current runningGas
        * the totalGas is used to determine our first assumption
        * the runningGas is what helps us determine if the current gas station start is good enough to run the entire circuit
    - if we reach a gas station that cannot supply us enough gas to get to the next one, we then make that one our start
        * so if (runningGas + currCost) < currCost, where currcost = gas[i] - cost[i], then we update the startIndex
        * we keep doing this until the loop ends
    - this is similar to Maximum Subarray b/c we know that if we have an array with all negative numbers in the first half and all positive in the second half, we only care about the second half
        * e.g. [-1, -2, -3, 3, 2, 1], we only care about [3, 2, 1] so we start there
        * the same can be said about this algorithm
        * why should we care about indices that we cannot reach when there are more in the future that possibly can?
    - how did we arrive at this solution?
        * we created visualized an array that represents the net cost of all gas stations at i
        * for example:
            - gas:  [ 1,   2,  3,  4,  5]
            - cost: [ 3,   4,  5,  1,  2]
            - net:  [-2, -2,  -2,  3,  3]
            - if we look at the net here, we see that it's pretty similar to our example above for Maximum Subarray

In [3]:
/**
 * @param {number[]} gas
 * @param {number[]} cost
 * @return {number}
 */
var canCompleteCircuit = function(gas, cost) {
    const _canCompleteCircuit = (startIndex, currIndex, currGas) => {
        if (currGas < 0) return false;
        if (startIndex === currIndex) return true;

        const newGas = currGas - cost[currIndex] + gas[currIndex];
        const newIndex = (currIndex + 1) === gas.length ? 0 : currIndex + 1;
        return _canCompleteCircuit(startIndex, newIndex, newGas);
    }

    for (let i = 0; i < gas.length; i++) {
        if (gas[i] < cost[i]) continue;
        const currGas = gas[i] - cost[i];
        if (_canCompleteCircuit(i, (i + 1) === gas.length ? 0 : i + 1, currGas)) {
            return i;
        }
    }

    return -1;
};

// two loop
var canCompleteCircuit = function(gas, cost) {
    // first loop to determine if net gas - cost >= 0
    let netGas = 0;
    for (let i = 0; i < gas.length; i++) {
        netGas += gas[i] - cost[i];
    }

    if (netGas < 0) return -1;

    // second loop to determine which index
    // if current cost > 0, continue at that start index
    // else, switch to the new index
    // but that index must ALSO meet the condition of gas[i] - cost[i] >= 0
    let runningGas = gas[0] - cost[0];
    let startIndex = 0;
    for (let i = 1; i < gas.length; i++) {
        const currCost = gas[i] - cost[i];
        if ((runningGas + currCost) < currCost) {
            startIndex = i;
            runningGas = currCost;
        }
        else {
            runningGas += currCost;
        }
    }

    return startIndex;
}

// one loop
var canCompleteCircuit = function(gas, cost) {
    // similar to Kadane's algorithm/sliding window
    // if our gas runs out, just start at another station
    let netGas = gas[0] - cost[0];
    let runningGas = gas[0] - cost[0];
    let startIndex = 0;
    for (let i = 1; i < gas.length; i++) {
        const currCost = gas[i] - cost[i];
        if ((runningGas + currCost) < currCost) {
            startIndex = i;
            runningGas = currCost;
        }
        else {
            runningGas += currCost;
        }
        netGas += currCost;
    }

    return netGas < 0 ? -1 : startIndex;
}